In [1]:
import json
import numpy as np
from graphviz import Digraph

def linear(x):
    return x

def relu(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def forward_propagation(model, input_data, weights):
    layers = model["layers"]
    output_layer = [input_data]

    for i, layer in enumerate(layers):
        activation_function = layer["activation_function"]
        weight_matrix = weights[i]
        bias = layer.get("bias", 1)

        input_activation = output_layer[-1]
        # print("Dimensi input_activation:", input_activation.shape)
        # print(input_activation)
        if len(input_activation.shape) == 1:
            input_activation_with_bias = np.insert(input_activation, 0, bias)
            input_activation_with_bias = input_activation_with_bias.reshape(1, -1)
        else:
            batch_size = input_activation.shape[0]
            bias_vector = np.ones((batch_size, 1)) * bias
            input_activation_with_bias = np.concatenate((bias_vector, input_activation), axis=1)
        # print("Dimensi input_activation_with_bias:", input_activation_with_bias.shape)
        # print(input_activation_with_bias)
        # print("Dimensi weight_matrix:", weight_matrix.shape)
        # print(weight_matrix)
        
        output_linear_combination = np.dot(input_activation_with_bias, weight_matrix)

        if activation_function == "linear":
            activation_result = linear(output_linear_combination)
        elif activation_function == "relu":
            activation_result = relu(output_linear_combination)
        elif activation_function == "sigmoid":
            activation_result = sigmoid(output_linear_combination)
        elif activation_function == "softmax":
            activation_result = softmax(output_linear_combination)

        output_layer.append(activation_result)

    return output_layer[-1]

In [76]:
# Load JSON input
with open("test/multilayer.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
input_data = np.array(data["case"]["input"])
weights = [np.array(layer_weights) for layer_weights in data["case"]["weights"]]
expected_output = np.array(data["expect"]["output"])

In [77]:
# Perform forward propagation
output = forward_propagation(model, input_data, weights)

# Check error
error = np.sum((output - expected_output) ** 2)

# Check if error is within tolerance
max_sse = data["expect"]["max_sse"]
if error <= max_sse:
    print("Output matches the expected output within tolerance. With error of", error, "which is less than or equal to", max_sse, ".")
else:
    print("Output does not match the expected output within tolerance. With error of", error, "which is greater than", max_sse, ".")

print("Output:")
print(output)

Output matches the expected output within tolerance. With error of 3.1555532735024718e-18 which is less than or equal to 1e-06 .
Output:
[[0.4846748]]


In [78]:
#Visualize the model
def visualize(data):
    dot = Digraph(comment='FFNN')
    dot.attr(rankdir='LR')
    # dot.attr(splines='false')
    dot.attr(rank='same')

    len_input = data["case"]["model"]["input_size"]
    len_layers = len(data["case"]["model"]["layers"])
    len_output = data["case"]["model"]["layers"][len_layers-1]["number_of_neurons"]
    print(len_input,len_layers,len_output)


    # no hidden layer, connect input directly to output
    if (len_layers == 1):
        for i in range(len_input+1):
            for j in range(len_output):
                weight = data["case"]["weights"][0][i][j]
                if (i==0): dot.edge(f'b', f'o{j+1}', label=f'w-b-0{j+1}={weight}')
                else: dot.edge(f'i{i}', f'o{j+1}', label=f'w-i{i}-o{j+1}={weight}')
    else:
        # Add input node
        for i in range(len_input+1):
            if(i==0): dot.node(f'b0', f'b0')
            else: dot.node(f'i{i}', f'i{i}')

        # Add output nodes
        for j in range(len_output):
            dot.node(f'o{j+1}', f'o{j+1}')

        # Process hidden layers
        for layer in range(len_layers):
            num_neurons = data["case"]["model"]["layers"][layer]["number_of_neurons"]
            
            #Add layer nodes
            if(layer!=len_layers-1):
                for neuron in range(num_neurons+1):
                    if (neuron==0): dot.node(f'b{layer+1}', f'b{layer+1}')
                    else: dot.node(f'h{layer+1}{neuron}', f'h{layer+1}{neuron}')

            if layer == 0:
                # Connect input layer to first hidden layer
                for i in range(len_input+1):
                    for neuron in range(num_neurons):
                        weight = data["case"]["weights"][layer][i][neuron]
                        if (i==0): dot.edge(f'b0', f'h{layer+1}{neuron+1}', label=f'w-b0-h{layer+1}{neuron+1}={weight}')
                        else: dot.edge(f'i{i}', f'h{layer+1}{neuron+1}', label=f'w-i{i}-h{layer+1}{neuron+1}={weight}')
            else:
                # Connect layers between them
                prev_num_neurons = data["case"]["model"]["layers"][layer-1]["number_of_neurons"]
                for p in range(prev_num_neurons+1):
                    for n in range(num_neurons):
                        weight = data["case"]["weights"][layer][p][n]
                        if (layer!=len_layers-1):
                            if (p==0): dot.edge(f'b{layer}', f'h{layer+1}{n+1}', label=f'w-b{layer}-h{layer+1}{n+1}={weight}')
                            else: dot.edge(f'h{layer}{p}', f'h{layer+1}{n+1}', label=f'w-h{layer}{p}-h{layer+1}{n+1}={weight}')
                        else:
                            if (p==0): dot.edge(f'b{layer}', f'o{n+1}', label=f'w-b{layer}-o{n+1}={weight}')
                            else: dot.edge(f'h{layer}{p}', f'o{n+1}', label=f'w-h{layer}{p}-o{n+1}={weight}')
    return dot

dot = visualize(data)
dot.render('output/multilayer', format='png', cleanup=True)



3 4 1


'output\\multilayer.png'